# **Kulik descriptor**

**Author:** Raissa Lohanna

**Problem class:** Regression

**Objective:** Generate Kulik's descriptors for the PubChemQC dataset after EDA and cleaning process

## Importing libraries and configuration

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from rdkit import Chem
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
tqdm.pandas()

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
sns.set_theme(style="darkgrid", palette="husl", rc={"figure.figsize":(10, 5)})

## Importing the dataset

In [4]:
df = pd.read_parquet("joined_data.parquet")

In [5]:
df.head()

,isomeric_smiles,charge,total_dipole_moment,multiplicity,homo,lumo,gap,total_energy,TD_ET_00,TD_OS_00,TD_ET_01,TD_OS_01,TD_ET_02,TD_OS_02,TD_ET_03,TD_OS_03,TD_ET_04,TD_OS_04,TD_ET_05,TD_OS_05,TD_ET_06,TD_OS_06,TD_ET_07,TD_OS_07,TD_ET_08,TD_OS_08,TD_ET_09,TD_OS_09
cid,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,0,12.362788,1,-4.655868,-0.302046,4.353822,-19287.579176,32381.393970,0.002393,33414.775701,0.002229,35041.644378,0.000038,35966.609062,0.004932,37121.320776,0.000746,37641.451596,0.006370,37976.268471,0.001117,38311.230525,0.001396,40062.494092,0.003212,40525.343416,0.002180
2,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,1,5.855433,1,-11.066870,-3.551086,7.515785,-19299.202955,46441.291104,0.000402,46825.670748,0.000412,56096.054110,0.000646,56938.129196,0.022929,57372.999148,0.000343,58598.534406,0.005652,59362.010763,0.005563,60451.940025,0.002308,62136.396688,0.025338,62379.605110,0.017436
3,C1=CC(C(C(=C1)C(=O)O)O)O,0,5.266252,1,-6.821894,-2.239497,4.582397,-15575.874068,32576.410766,0.053438,34937.703709,0.127298,35191.469929,0.035610,38959.724483,0.045520,45869.452080,0.002901,46837.502902,0.004995,48214.775245,0.008408,48851.170886,0.003279,50890.898644,0.046370,51729.941085,0.009966
4,CC(CN)O,0,2.681395,1,-6.187869,1.847653,8.035522,-6794.535860,45133.600212,0.003867,50498.445390,0.022722,52102.569231,0.001101,53565.199229,0.044094,54291.396641,0.024346,58571.208342,0.009868,60147.530252,0.014954,60319.624771,0.008397,60994.089781,0.004093,62458.026397,0.000847
5,C(C(=O)COP(=O)(O)O)N,0,8.447997,1,-7.270882,-1.529280,5.741602,-24256.843704,34507.648887,0.001015,41628.443712,0.008713,45462.755074,0.001559,46943.403498,0.000598,48366.754296,0.003638,50735.499802,0.000874,53071.087855,0.009599,54805.905776,0.025285,54979.815042,0.000488,55740.210361,0.013173


## Applying the Kulik's decriptor

Equação (https://pubs.acs.org/doi/full/10.1021/ja401184g):

$P_d = \sum_i \sum_j P_i P_j \delta(d - d_{ij})$


Importing the elements database

In [7]:
elem_data = pd.read_csv('Atomic properties DB.csv')

In [8]:
elem_data.head()

,Z,Name,Symbol,Average atomic mass,Electronegativity (Pauling),First Ionization Energy (eV),Atomic Radii (pm),Van der Waals Radii (pm),Covalent Radii (pm),Valence electrons
0,1,Hydrogen,H,1007,2.2,1.359844e+06,25,120,38,1.0
1,2,Helium,He,4.002602(2),—,2.458741e+06,31,140,32,2.0
2,3,Lithium,Li,6.941(2),0.98,5.391720e+05,145,182,134,1.0
3,4,Beryllium,Be,9.012182(3),1.57,9.322700e+00,105,—,90,2.0
4,5,Boron,B,10.811(7),02.04,8.298030e+05,85,—,82,3.0


In [9]:
elem_data.dtypes

Z                                 int64
Name                             object
Symbol                           object
Average atomic mass              object
Electronegativity (Pauling)      object
First Ionization Energy (eV)    float64
Atomic Radii (pm)                object
Van der Waals Radii (pm)         object
Covalent Radii (pm)               int64
Valence electrons               float64
dtype: object

Fixing null data and type of objects

In [10]:
elem_data['Electronegativity (Pauling)'] = np.where( 
    ( elem_data['Electronegativity (Pauling)']=='—'), np.NaN, elem_data['Electronegativity (Pauling)'])

elem_data['Atomic Radii (pm)'] = np.where( 
    ( elem_data['Atomic Radii (pm)']=='—'), np.NaN, elem_data['Atomic Radii (pm)'])

elem_data['Van der Waals Radii (pm)'] = np.where( 
    ( elem_data['Van der Waals Radii (pm)']=='—'), np.NaN, elem_data['Van der Waals Radii (pm)'])

elem_data['Valence electrons'] = np.where( 
    ( elem_data['Valence electrons']=='—'), np.NaN, elem_data['Valence electrons'])

In [11]:
float_data = ['Electronegativity (Pauling)', 'Atomic Radii (pm)', 'Van der Waals Radii (pm)']

elem_data[float_data] = elem_data[float_data].astype(float)

In [12]:
elem_data.dtypes

Z                                 int64
Name                             object
Symbol                           object
Average atomic mass              object
Electronegativity (Pauling)     float64
First Ionization Energy (eV)    float64
Atomic Radii (pm)               float64
Van der Waals Radii (pm)        float64
Covalent Radii (pm)               int64
Valence electrons               float64
dtype: object

In [13]:
#Converting the df into a dictionary to be used in the Kulik's descriptor

'''
    Converting DataFrame to Dicionary using a COLUMN as KEY, in the format:
    
    prop: {<ELEMENT_1>: (prop1, prop2, prop3,...),
            <ELEMENT_2>: (prop1, prop2, prop3,...),...
            }
    
    Lengh of list of properties: 10
    
    Index of each property:
    
        0: index
        1: Z
        2: Name
        3: Average atomic mass
        4: Electronegativity (Pauling)
        5: First Ionization Energy (eV)
        6: Atomic Radii (pm)
        7: Van der Waals Radii (pm)
        8: Covalent Radii (pm)
        9: Valence electrons 
'''

properties_dict = elem_data.set_index('Symbol').T.to_dict('list')

#### Applying the Kulik's descriptor

In [61]:
def create_property_descriptors(smiles, depth, prop, prop_index=0):
    '''
    Função:
        create_property_descriptors(smiles, depth, prop, prop_index=0)
    
    Descrição:
        Permite criar descritores baseados em propriedades atômicas e conectividade básica entre os átomos

    Argumentos:
        smiles:     string SMILES da molécula que se deseja gerar os descritores. Exemplo: 'C(C)CO'.
        depth:      profundidade, parâmetro que permite definir o nivel máximo de conectividade que será 
                    usado na geração dos descritores. Exemplo, se depth=3, três descritores serão gerados,
                    com conectividade iguais a um, dois e três.
        prop:       dicionário formado por pares 'átomo': propriedade ou 'átomo':[propriedades] ou 
                    'átomo':(propriedades).
        prop_index: índice da propriedade de interesse nos valores de 'prop'. Se estes valores não forem
                    listas ou tuplas, este parâmetro é ignorado. Se forem, este parâmetro deve ser
                    configurado com um número maior ou igual a zero e menor do que o número de itens nas
                    listas ou tuplas. O valor default deste parâmetro é zero, ou seja, a função pegará a
                    primeira propriedade da lista.
    '''
    try:
        # Criar a estrutura a partir do SMILES da molécula, adicionando os H faltantes
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        
        # Preparar listas de resultados (separação por depth=0, 1, 2, 3...)
        result_list = [0] * (depth+1) # Se depth = 2, precisamos de 3 itens (para d=[0,1,2])
            
        # Obter a matriz de distâncias entre cada um dos átomos e a lista de átomos
        atom_list = [ atom.GetSymbol() for atom in mol.GetAtoms() ]
        n_atoms = len(atom_list)
        dist_matrix = np.tril(Chem.rdmolops.GetDistanceMatrix(mol, force=True))
            
        # Iterar toda a matriz, extraindo os dados de interesse e separando por valor de depth
        for i in range(0, n_atoms):
            for j in range(0, n_atoms):
                d = int(dist_matrix[i][j])
                
                if d <= depth and d == 0 and i == j:
                    a_i = atom_list[i]
                    p_i = 0
                    
                    if prop[a_i] is list or tuple:  
                        p_i = prop[a_i][prop_index]   # caso haja + de 1 propriedade p/ calcular
                        
                    else:
                        p_i = prop[a_i]            # caso haja somente 1 propriedade p/ calcular
                        
                    result_list[d] += round(p_i * p_i, 3)
                
                if d <= depth and d > 0:
                    a_i = atom_list[i]
                    a_j = atom_list[j]
                    p_i, p_j = 0, 0
                    
                    if prop[a_i] is list or tuple:  
                        p_i = prop[a_i][prop_index]   # caso haja + de 1 propriedade p/ calcular
                    else:
                        p_i = prop[a_i]            # caso haja somente 1 propriedade p/ calcular
                    if prop[a_j] is list or tuple:
                        p_j = prop[a_j][prop_index]
                    else:
                        p_j = prop[a_j]
                    result_list[d] += round(p_i * p_j, 3)    # posição [d] na lista de resultados é incrementada
        
        # Finalizando
        return result_list

    except TypeError:

        #return smiles
        return print(TypeError)

In [ ]:
from math import sqrt
from joblib import Parallel, delayed

**Property 3: Electronegativity (Pauling)**

In [ ]:
df['Kulik_electronegativity'] = Parallel(n_jobs=-1, prefer="threads", verbose=10)(
    delayed(create_property_descriptors)
    (smiles=x, depth=3, prop=properties_dict, prop_index=3) 
    for x in df['isomeric_smiles']
    )

**Property 4: 1st Ionization Energy**

In [ ]:
df['Kulik_1st_ionization_energy'] = Parallel(n_jobs=-1, prefer="threads", verbose=10)(
    delayed(create_property_descriptors)
    (smiles=x, depth=3, prop=properties_dict, prop_index=4) 
    for x in df['isomeric_smiles']
    )

**Property 5: 1st Atomic Radii**

In [ ]:
df['Kulik_atomic_radii'] = Parallel(n_jobs=-1, prefer="threads", verbose=10)(
    delayed(create_property_descriptors)
    (smiles=x, depth=3, prop=properties_dict, prop_index=5) 
    for x in df['isomeric_smiles']
    )

**Property 6: Van Der Waals Radii**

In [ ]:
df['Kulik_van_der_waals_radii'] = Parallel(n_jobs=-1, prefer="threads", verbose=10)(
    delayed(create_property_descriptors)
    (smiles=x, depth=3, prop=properties_dict, prop_index=6) 
    for x in df['isomeric_smiles']
    )

**Property 7: Covalent Radii**

In [ ]:
df['Kulik_covalent_radii'] = Parallel(n_jobs=-1, prefer="threads", verbose=10)(
    delayed(create_property_descriptors)
    (smiles=x, depth=3, prop=properties_dict, prop_index=7) 
    for x in df['isomeric_smiles']
    )

**Property 8: valence_ealence Electrons**

In [ ]:
df['Kulik_valence_e'] = Parallel(n_jobs=-1, prefer="threads", verbose=10)(
    delayed(create_property_descriptors)
    (smiles=x, depth=3, prop=properties_dict, prop_index=8) 
    for x in df['isomeric_smiles']
    )

In [8]:
df.head()

,isomeric_smiles,charge,total_dipole_moment,multiplicity,homo,lumo,gap,total_energy,TD_ET_00,TD_OS_00,TD_ET_01,TD_OS_01,TD_ET_02,TD_OS_02,TD_ET_03,TD_OS_03,TD_ET_04,TD_OS_04,TD_ET_05,TD_OS_05,TD_ET_06,TD_OS_06,TD_ET_07,TD_OS_07,TD_ET_08,TD_OS_08,TD_ET_09,TD_OS_09,Kulik_electronegativity,Kulik_1st_ionization_energy,Kulik_atomic_radii,Kulik_van_der_waals_radii,Kulik_covalent_radii,Kulik_valence_e
cid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,0,12.362788,1,-4.655868,-0.302046,4.353822,-19287.579176,32381.393970,0.002393,33414.775701,0.002229,35041.644378,0.000038,35966.609062,0.004932,37121.320776,0.000746,37641.451596,0.006370,37976.268471,0.001117,38311.230525,0.001396,40062.494092,0.003212,40525.343416,0.002180,"[197.37600000000003, 196.24600000000015, 353.1...","[40966461556793.15, 402444012.0230001, 5319019...","[73350.0, 88550.0, 128425.0, 152925.0]","[621341.0, 697000.0, 1136538.0, 1482730.0]","[104850, 124663, 190046, 236813]","[330.0, 332.0, 499.0, 507.0]"
2,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,1,5.855433,1,-11.066870,-3.551086,7.515785,-19299.202955,46441.291104,0.000402,46825.670748,0.000412,56096.054110,0.000646,56938.129196,0.022929,57372.999148,0.000343,58598.534406,0.005652,59362.010763,0.005563,60451.940025,0.002308,62136.396688,0.025338,62379.605110,0.017436,"[202.21600000000004, 203.81400000000016, 358.7...","[42815637261129.15, 1852246162276.0237, 531902...","[73975.0, 90050.0, 130175.0, 156175.0]","[635741.0, 715240.0, 1156938.0, 1521370.0]","[106294, 127437, 192972, 242513]","[331.0, 338.0, 503.0, 517.0]"
3,C1=CC(C(C(=C1)C(=O)O)O)O,0,5.266252,1,-6.821894,-2.239497,4.582397,-15575.874068,32576.410766,0.053438,34937.703709,0.127298,35191.469929,0.035610,38959.724483,0.045520,45869.452080,0.002901,46837.502902,0.004995,48214.775245,0.008408,48851.170886,0.003279,50890.898644,0.046370,51729.941085,0.009966,"[131.57000000000002, 131.356, 204.548000000000...","[22211467962119.56, 5555669054312.931, 5558494...","[53700.0, 64350.0, 93750.0, 105400.0]","[409916.0, 462380.0, 711024.0, 906324.0]","[74371, 86939, 130073, 155065]","[264.0, 246.0, 372.0, 430.0]"
4,CC(CN)O,0,2.681395,1,-6.187869,1.847653,8.035522,-6794.535860,45133.600212,0.003867,50498.445390,0.022722,52102.569231,0.001101,53565.199229,0.044094,54291.396641,0.024346,58571.208342,0.009868,60147.530252,0.014954,60319.624771,0.008397,60994.089781,0.004093,62458.026397,0.000847,"[84.14200000000002, 84.13199999999999, 133.042...","[20609509176436.383, 5804799907079.712, 150507...","[28150.0, 33800.0, 43225.0, 39575.0]","[263429.0, 287830.0, 438370.0, 487310.0]","[41737, 49284, 67900, 68744]","[118.0, 116.0, 145.0, 131.0]"
5,C(C(=O)COP(=O)(O)O)N,0,8.447997,1,-7.270882,-1.529280,5.741602,-24256.843704,34507.648887,0.001015,41628.443712,0.008713,45462.755074,0.001559,46943.403498,0.000598,48366.754296,0.003638,50735.499802,0.000874,53071.087855,0.009599,54805.905776,0.025285,54979.815042,0.000488,55740.210361,0.013173,"[131.43400000000003, 119.38799999999999, 203.4...","[27278102470205.383, 13368963280041.027, 27183...","[51925.0, 60000.0, 74275.0, 68850.0]","[373845.0, 400550.0, 584474.0, 612574.0]","[72845, 82779, 109895, 106353]","[286.0, 258.0, 403.0, 300.0]"


In [22]:
df.dtypes

isomeric_smiles                 object
charge                           int64
total_dipole_moment            float64
multiplicity                     int64
homo                           float64
lumo                           float64
gap                            float64
total_energy                   float64
TD_ET_00                       float64
TD_OS_00                       float64
TD_ET_01                       float64
TD_OS_01                       float64
TD_ET_02                       float64
TD_OS_02                       float64
TD_ET_03                       float64
TD_OS_03                       float64
TD_ET_04                       float64
TD_OS_04                       float64
TD_ET_05                       float64
TD_OS_05                       float64
TD_ET_06                       float64
TD_OS_06                       float64
TD_ET_07                       float64
TD_OS_07                       float64
TD_ET_08                       float64
TD_OS_08                 

Atomic_radii

In [30]:
df_atomic_radii = df.Kulik_atomic_radii.apply(pd.Series)
df_atomic_radii = df_atomic_radii.rename(columns={0:'atomic_radii_lvl0', 1: 'atomic_radii_lvl1', 2: 'atomic_radii_lvl2', 3:'atomic_radii_lvl3'})

In [53]:
df_atomic_radii.reset_index(inplace=True)

In [54]:
df_atomic_radii.head()

,cid,atomic_radii_lvl0,atomic_radii_lvl1,atomic_radii_lvl2,atomic_radii_lvl3
0,1,73350.0,88550.0,128425.0,152925.0
1,2,73975.0,90050.0,130175.0,156175.0
2,3,53700.0,64350.0,93750.0,105400.0
3,4,28150.0,33800.0,43225.0,39575.0
4,5,51925.0,60000.0,74275.0,68850.0


Van der waals radii

In [32]:
df_van_der_waals_radii = df.Kulik_van_der_waals_radii.apply(pd.Series)
df_van_der_waals_radii = df_van_der_waals_radii.rename(columns={0:'van_der_waals_radii_lvl0', 1: 'van_der_waals_radii_lvl1', 2: 'van_der_waals_radii_lvl2', 3:'van_der_waals_radii_lvl3'})

In [51]:
df_van_der_waals_radii.reset_index(inplace=True)

In [52]:
df_van_der_waals_radii.head()

,cid,van_der_waals_radii_lvl0,van_der_waals_radii_lvl1,van_der_waals_radii_lvl2,van_der_waals_radii_lvl3
0,1,621341.0,697000.0,1136538.0,1482730.0
1,2,635741.0,715240.0,1156938.0,1521370.0
2,3,409916.0,462380.0,711024.0,906324.0
3,4,263429.0,287830.0,438370.0,487310.0
4,5,373845.0,400550.0,584474.0,612574.0


Covalent radii

In [34]:
df_covalent_radii = df.Kulik_covalent_radii.apply(pd.Series)
df_covalent_radii = df_covalent_radii.rename(columns={0:'covalent_radii_lvl0', 1: 'covalent_radii_lvl1', 2: 'covalent_radii_lvl2', 3:'covalent_radii_lvl3'})

In [49]:
df_covalent_radii.reset_index(inplace=True)

In [50]:
df_covalent_radii.head()

,cid,covalent_radii_lvl0,covalent_radii_lvl1,covalent_radii_lvl2,covalent_radii_lvl3
0,1,104850.0,124663.0,190046.0,236813.0
1,2,106294.0,127437.0,192972.0,242513.0
2,3,74371.0,86939.0,130073.0,155065.0
3,4,41737.0,49284.0,67900.0,68744.0
4,5,72845.0,82779.0,109895.0,106353.0


Valence electron

In [36]:
df_valence_e = df.Kulik_valence_e.apply(pd.Series)
df_valence_e = df_valence_e.rename(columns={0:'valence_e_lvl0', 1: 'valence_e_lvl1', 2: 'valence_e_lvl2', 3:'valence_e_lvl3'})

In [47]:
df_valence_e.reset_index(inplace=True)

In [48]:
df_valence_e.head()

,cid,valence_e_lvl0,valence_e_lvl1,valence_e_lvl2,valence_e_lvl3
0,1,330.0,332.0,499.0,507.0
1,2,331.0,338.0,503.0,517.0
2,3,264.0,246.0,372.0,430.0
3,4,118.0,116.0,145.0,131.0
4,5,286.0,258.0,403.0,300.0


1st_ionization_energy

In [38]:
df_1st_ionization_energy = df.Kulik_1st_ionization_energy.apply(pd.Series)
df_1st_ionization_energy = df_1st_ionization_energy.rename(columns={0:'1st_ionization_energy_lvl0', 1: '1st_ionization_energy_lvl1', 2: '1st_ionization_energy_lvl2', 3:'1st_ionization_energy_lvl3'})

In [43]:
df_1st_ionization_energy.reset_index(inplace=True)

In [44]:
df_1st_ionization_energy.head()

,cid,1st_ionization_energy_lvl0,1st_ionization_energy_lvl1,1st_ionization_energy_lvl2,1st_ionization_energy_lvl3
0,1,4.096646e+13,4.024440e+08,5.319019e+13,3.727887e+13
1,2,4.281564e+13,1.852246e+12,5.319021e+13,3.913073e+13
2,3,2.221147e+13,5.555669e+12,5.558494e+12,2.035734e+13
3,4,2.060951e+13,5.804800e+12,1.505076e+13,3.170681e+13
4,5,2.727810e+13,1.336896e+13,2.718336e+13,3.260116e+13


Electronegativity

In [40]:
df_electronegativity = df.Kulik_electronegativity.apply(pd.Series)
df_electronegativity = df_electronegativity.rename(columns={0:'electronegativity_lvl0', 1: 'electronegativity_lvl1', 2: 'electronegativity_lvl2', 3:'electronegativity_lvl3'})

In [45]:
df_electronegativity.reset_index(inplace=True)

In [46]:
df_electronegativity.head()

,cid,electronegativity_lvl0,electronegativity_lvl1,electronegativity_lvl2,electronegativity_lvl3
0,1,197.376,196.246,353.176,449.602
1,2,202.216,203.814,358.786,462.780
2,3,131.570,131.356,204.548,277.974
3,4,84.142,84.132,133.042,161.628
4,5,131.434,119.388,203.486,209.052


Joining dfs:

In [59]:
df_kulik = (
    df_atomic_radii.merge(
        df_van_der_waals_radii.merge(
            df_covalent_radii.merge(
                df_valence_e.merge(
                    df_1st_ionization_energy.merge(df_electronegativity))))))

In [60]:
df_kulik.shape

(3332473, 25)

In [62]:
df.head()

,isomeric_smiles,charge,total_dipole_moment,multiplicity,homo,lumo,gap,total_energy,TD_ET_00,TD_OS_00,TD_ET_01,TD_OS_01,TD_ET_02,TD_OS_02,TD_ET_03,TD_OS_03,TD_ET_04,TD_OS_04,TD_ET_05,TD_OS_05,TD_ET_06,TD_OS_06,TD_ET_07,TD_OS_07,TD_ET_08,TD_OS_08,TD_ET_09,TD_OS_09,Kulik_electronegativity,Kulik_1st_ionization_energy,Kulik_atomic_radii,Kulik_van_der_waals_radii,Kulik_covalent_radii,Kulik_valence_e
cid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,0,12.362788,1,-4.655868,-0.302046,4.353822,-19287.579176,32381.393970,0.002393,33414.775701,0.002229,35041.644378,0.000038,35966.609062,0.004932,37121.320776,0.000746,37641.451596,0.006370,37976.268471,0.001117,38311.230525,0.001396,40062.494092,0.003212,40525.343416,0.002180,"[197.37600000000003, 196.24600000000015, 353.1...","[40966461556793.15, 402444012.0230001, 5319019...","[73350.0, 88550.0, 128425.0, 152925.0]","[621341.0, 697000.0, 1136538.0, 1482730.0]","[104850, 124663, 190046, 236813]","[330.0, 332.0, 499.0, 507.0]"
2,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,1,5.855433,1,-11.066870,-3.551086,7.515785,-19299.202955,46441.291104,0.000402,46825.670748,0.000412,56096.054110,0.000646,56938.129196,0.022929,57372.999148,0.000343,58598.534406,0.005652,59362.010763,0.005563,60451.940025,0.002308,62136.396688,0.025338,62379.605110,0.017436,"[202.21600000000004, 203.81400000000016, 358.7...","[42815637261129.15, 1852246162276.0237, 531902...","[73975.0, 90050.0, 130175.0, 156175.0]","[635741.0, 715240.0, 1156938.0, 1521370.0]","[106294, 127437, 192972, 242513]","[331.0, 338.0, 503.0, 517.0]"
3,C1=CC(C(C(=C1)C(=O)O)O)O,0,5.266252,1,-6.821894,-2.239497,4.582397,-15575.874068,32576.410766,0.053438,34937.703709,0.127298,35191.469929,0.035610,38959.724483,0.045520,45869.452080,0.002901,46837.502902,0.004995,48214.775245,0.008408,48851.170886,0.003279,50890.898644,0.046370,51729.941085,0.009966,"[131.57000000000002, 131.356, 204.548000000000...","[22211467962119.56, 5555669054312.931, 5558494...","[53700.0, 64350.0, 93750.0, 105400.0]","[409916.0, 462380.0, 711024.0, 906324.0]","[74371, 86939, 130073, 155065]","[264.0, 246.0, 372.0, 430.0]"
4,CC(CN)O,0,2.681395,1,-6.187869,1.847653,8.035522,-6794.535860,45133.600212,0.003867,50498.445390,0.022722,52102.569231,0.001101,53565.199229,0.044094,54291.396641,0.024346,58571.208342,0.009868,60147.530252,0.014954,60319.624771,0.008397,60994.089781,0.004093,62458.026397,0.000847,"[84.14200000000002, 84.13199999999999, 133.042...","[20609509176436.383, 5804799907079.712, 150507...","[28150.0, 33800.0, 43225.0, 39575.0]","[263429.0, 287830.0, 438370.0, 487310.0]","[41737, 49284, 67900, 68744]","[118.0, 116.0, 145.0, 131.0]"
5,C(C(=O)COP(=O)(O)O)N,0,8.447997,1,-7.270882,-1.529280,5.741602,-24256.843704,34507.648887,0.001015,41628.443712,0.008713,45462.755074,0.001559,46943.403498,0.000598,48366.754296,0.003638,50735.499802,0.000874,53071.087855,0.009599,54805.905776,0.025285,54979.815042,0.000488,55740.210361,0.013173,"[131.43400000000003, 119.38799999999999, 203.4...","[27278102470205.383, 13368963280041.027, 27183...","[51925.0, 60000.0, 74275.0, 68850.0]","[373845.0, 400550.0, 584474.0, 612574.0]","[72845, 82779, 109895, 106353]","[286.0, 258.0, 403.0, 300.0]"


Saving data:

In [61]:
df.to_parquet("../Datasets/Kulik_descriptors.parquet")